# DreamBooth

In [1]:
#@title One-Click Setup

# remove sample data provided by Google Colab
!rm -r /content/sample_data

# download necessary files
!git clone https://github.com/rmyj/dreambooth.git

!pip install -r /content/dreambooth/requirements.txt

# download validation set - designed for Google Colab
%cd /content/
for i in range(173):
  i = i + 1
  url = f"https://github.com/rmyj/txt2img/raw/main/assets/sd-v1-4.zip.{i:03}"
  !wget $url
# concatenate all part files to one zip
!cat /content/sd-v1-4.zip* > /content/sd-v1-4.zip
!rm /content/sd-v1-4.zip.*
# repair combination
!zip -FF /content/sd-v1-4.zip --out /content/sd-v1-4-full.zip
!rm /content/sd-v1-4.zip
# unzip combination
!unzip /content/sd-v1-4-full.zip
!rm /content/sd-v1-4-full.zip

Cloning into 'dreambooth'...
remote: Enumerating objects: 13643, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 13643 (delta 114), reused 136 (delta 51), pack-reused 13380
Receiving objects: 100% (13643/13643), 10.70 MiB | 18.92 MiB/s, done.
Resolving deltas: 100% (9353/9353), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 191 kB 16.1 MB/s 
     |████████████████████████████████| 62.5 MB 1.2 MB/s 
     |████████████████████████████████| 524 kB 73.1 MB/s 
     |████████████████████████████████| 5.8 MB 61.0 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 182 kB 72.4 MB/s 
     |████████████████████████████████| 7.6 MB 68.4 MB/s 
     |████████████████████████████████| 1.6 MB 66.4 MB/s 
/content
--2023-01-03 06:04:36--  https://github.com/rmyj/txt2img/raw/main/assets/s

In [ ]:
#@title Upload photos of your subject
from google.colab import files
files.upload()

!mkdir /content/dreambooth/data/
!mkdir /content/dreambooth/data/subject/

!mkdir /content/dreambooth/output/

!mv /content/*.PNG /content/dreambooth/data/subject/
!mv /content/*.png /content/dreambooth/data/subject/
!mv /content/*.jpg /content/dreambooth/data/subject/

In [ ]:
#@title Train
!accelerate launch /content/dreambooth/scripts/train_dreambooth.py \
  --pretrained_model_name_or_path="/content/sd-v1-4/"  \
  --instance_data_dir="/content/dreambooth/data/subject/" \
  --output_dir="/content/dreambooth/models/output" \
  --instance_prompt="a photo of fefefefefefefe person" \
  --resolution=512 \
  --train_batch_size=1 \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=2000

In [ ]:
#@title Save
#@markdown This code block requests permission for this notebook to access your Google Drive, then zips the output folder, and moves it to your Google Drive home directory for later download. This method is by far the fastest way to download files from Google Colab, as the direct browser download takes a very long time.
#@markdown output.zip will be saved in `/content/drive/MyDrive/txt2img/output.zip`

from google.colab import drive
drive.mount('/content/drive')

!mkdir /content/drive/MyDrive/dreambooth

!zip -r /content/output.zip /content/dreambooth/models/output/

!mkdir /content/drive/MyDrive/dreambooth/

!rm /content/drive/MyDrive/dreambooth/output.zip

!mv /content/output.zip /content/drive/MyDrive/dreambooth/output.zip

## (optional) convert to .ckpt

In [ ]:
#@markdown Download script
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py

In [ ]:
#@markdown Run conversion.
input_path = "/home/jovyan/work/models/sd-v1-5-inpainting"
output_path = "/home/jovyan/work/models/sd-v1-5-inpainting.ckpt"

!python3 scripts/convert_diffusers_to_original_stable_diffusion.py --model_path $input_path --checkpoint_path $output_path
print(f"[*] Converted ckpt saved at {output_path}")

***
# Inference

## Setup

In [ ]:
from PIL import Image
from diffusers import StableDiffusionPipeline
import random
import uuid
import torch

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

# set variables for inference
model_dir = "/content/dreambooth/models/output"
pipe = StableDiffusionPipeline.from_pretrained(model_dir, torch_dtype=torch.float16).to("cuda")

# to disable the safety checker
pipe.safety_checker = lambda images, clip_input: (images, False)

## Generate

## 512x512 single image synthesis

In [ ]:
prompt = "oil painting of a fefefefefefefe person"
num_inference_steps=68
guidance_scale=7.5
width=512
height=512

seed = uuid.uuid4().int & (1<<64)-1

g_cuda = torch.Generator("cuda").manual_seed(seed)

print(f"{prompt}-{seed}")
image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
image.save(f"/home/jovyan/work/output/{prompt}-{seed}-md.png")
image